## 1. 分类练习
对census_income数据集进行预处理（包括但不限于填充缺失值、数据标准化、标称数据编码）。  
以最后一列“>50K”为正类，“<=50K”为负类，选择至少4中分类模型进行建模，并进行结果评估和比较。

http://archive.ics.uci.edu/ml/datasets/Census+Income 

In [8]:
from sklearn import preprocessing,datasets,linear_model,tree
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,accuracy_score, f1_score, precision_score, roc_auc_score,recall_score

In [9]:
data = pd.read_csv('adult.data',header = None,names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','kind'])
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,kind
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [10]:
#把？替换为空值
data = data.replace(' ?',np.nan)
data.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
kind                 0
dtype: int64

In [11]:
#删去空值
data = data.dropna()

In [12]:
#定量数据标准化
data_quantity = data[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']]
scaler = preprocessing.StandardScaler().fit(data_quantity)
quan_scaled = scaler.transform(data_quantity)

In [13]:
#定性数据标称特征编码，使用OneHotEncoder
workclass = [' Private',' Self-emp-not-inc',' Self-emp-inc',' Federal-gov',' Local-gov',' State-gov',' Without-pay',' Never-worked']
education =[' Bachelors',' Some-college',' 11th',' HS-grad',' Prof-school',' Assoc-acdm',' Assoc-voc',' 9th',' 7th-8th',' 12th',' Masters',' 1st-4th',' 10th',' Doctorate',' 5th-6th',' Preschool']
marital_status = [' Married-civ-spouse',' Divorced',' Never-married',' Separated',' Widowed',' Married-spouse-absent',' Married-AF-spouse']
occupation = [' Tech-support',' Craft-repair',' Other-service',' Sales',' Exec-managerial',' Prof-specialty',' Handlers-cleaners',' Machine-op-inspct',' Adm-clerical',' Farming-fishing',' Transport-moving',' Priv-house-serv',' Protective-serv',' Armed-Forces']
relationship = [' Wife',' Own-child',' Husband',' Not-in-family',' Other-relative',' Unmarried']
race = [' White',' Asian-Pac-Islander',' Amer-Indian-Eskimo',' Other',' Black']
sex = [' Female',' Male']
native_country = [' United-States',' Cambodia',' England',' Puerto-Rico',' Canada',' Germany',' Outlying-US(Guam-USVI-etc)',' India',' Japan',' Greece',' South',' China',' Cuba',' Iran',' Honduras',' Philippines',' Italy',' Poland',' Jamaica',' Vietnam',' Mexico',' Portugal',' Ireland',' France',' Dominican-Republic',' Laos',' Ecuador',' Taiwan',' Haiti',' Columbia',' Hungary',' Guatemala',' Nicaragua',' Scotland',' Thailand',' Yugoslavia',' El-Salvador',' Trinadad&Tobago',' Peru',' Hong',' Holand-Netherlands']
kind = [' <=50K',' >50K']
enc = preprocessing.OneHotEncoder(categories = [workclass,education,marital_status,occupation,relationship,race,sex,native_country,kind])
data_qualitative = data[['workclass','education','marital-status','occupation','relationship','race','sex','native-country','kind']]
enc.fit(data_qualitative)
quali_encoding = enc.transform(data_qualitative).toarray()

In [14]:
#将标称特征后的定性数据标准化
scaler = preprocessing.StandardScaler().fit(quali_encoding)
quali_scaled = scaler.transform(quali_encoding)

In [15]:
#把编码和标准化后的数据合并在一起
X = pd.merge(pd.DataFrame(quan_scaled),pd.DataFrame(quali_scaled),left_index = True, right_index = True)
X

,0_x,1_x,2_x,3_x,4_x,5_x,0_y,1_y,2_y,3_y,...,91,92,93,94,95,96,97,98,99,100
0,0.042796,-1.062722,1.128918,0.146092,-0.218586,-0.077734,-1.682144,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
1,0.880288,-1.007871,1.128918,-0.147445,-0.218586,-2.331531,-1.682144,3.327105,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
2,-0.033340,0.244693,-0.439738,-0.147445,-0.218586,-0.077734,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
3,1.108695,0.425240,-1.224066,-0.147445,-0.218586,-0.077734,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
4,-0.794697,1.406658,1.128918,-0.147445,-0.218586,-0.077734,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,-0.870832,0.638972,0.736754,-0.147445,-0.218586,-0.244682,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
30158,0.118931,-0.335252,-0.439738,-0.147445,-0.218586,-0.077734,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,-1.737042,1.737042
30159,1.489374,-0.358575,-0.439738,-0.147445,-0.218586,-0.077734,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691
30160,-1.251511,0.110705,-0.439738,-0.147445,-0.218586,-1.747213,0.594479,-0.300562,-0.192152,-0.179648,...,-0.019101,-0.023747,-0.023038,-0.057675,-0.024436,-0.031553,-0.025106,-0.005758,0.575691,-0.575691


In [16]:
#变成array的形式
X = np.array(X)

In [32]:
#划分训练集和测试集
X_train = X[:int(0.6*len(X)),:-2]
X_test = X[int(0.6*len(X)):,:-2]
Y_train = X[:int(0.6*len(X)), -1]#由于kind只有两个值，所以可以使用一位代替
Y_test = X[int(0.6*len(X)):, -1]

In [38]:
#最小二乘
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)
y_pred = regr.predict(X_test)
#print('Score: %.2f' % regr.score(X_train, Y_train))
print('Accuracy: %.2f'% accuracy_score(Y_test.astype('int32'), y_pred.astype('int32')))
print('Recall: %.2f'% recall_score(Y_test.astype('int32'), y_pred.astype('int'), average = 'micro'))
print('F1_score: %.2f'% f1_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Precision_score: %.2f' % precision_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Roc_auc_score: %.2f' % roc_auc_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))

Accuracy: 0.78
Recall: 0.78
F1_score: 0.78
Precision_score: 0.78
Roc_auc_score: 0.61


In [37]:
#逻辑回归
clf_l2_LR = LogisticRegression(penalty = 'l2', tol = 0.01, solver = 'saga')
clf_l2_LR.fit(X_train, Y_train.astype('int'))
y_pred = clf_l2_LR.predict(X_test)
print('Accuracy: %.2f'% accuracy_score(Y_test.astype('int32'), y_pred.astype('int32')))
print('Recall: %.2f'% recall_score(Y_test.astype('int'), y_pred.astype('int'), average = 'micro'))
print('F1_score: %.2f'% f1_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Precision_score: %.2f' % precision_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Roc_auc_score: %.2f' % roc_auc_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
#clf_l2_LR.score(X_train, Y_train.astype('int'))

Accuracy: 0.85
Recall: 0.85
F1_score: 0.85
Precision_score: 0.85
Roc_auc_score: 0.77


In [36]:
#决策树
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train.astype('int'))
y_pred = clf.predict(X_test)
print('Mean suqred error: %.2f'% mean_squared_error(Y_test, y_pred))
print('R2 score: %.2f'% r2_score(Y_test, y_pred))
print('Accuracy: %.2f'% accuracy_score(Y_test.astype('int32'), y_pred.astype('int32')))
print('Recall: %.2f'% recall_score(Y_test.astype('int'), y_pred.astype('int'), average = 'micro'))
print('F1_score: %.2f'% f1_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Precision_score: %.2f' % precision_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Roc_auc_score: %.2f' % roc_auc_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
#clf.score(X_train, Y_train.astype('int'))

Mean suqred error: 0.83
R2 score: 0.18
Accuracy: 0.81
Recall: 0.81
F1_score: 0.81
Precision_score: 0.81
Roc_auc_score: 0.75


In [35]:
#朴素贝叶斯
gnb = GaussianNB()
gnb.fit(X_train, Y_train.astype('int'))
y_pred = gnb.predict(X_test)
print('Accuracy: %.2f'% accuracy_score(Y_test.astype('int32'), y_pred.astype('int32')))
print('Recall: %.2f'% recall_score(Y_test.astype('int'), y_pred.astype('int'), average = 'micro'))
print('F1_score: %.2f'% f1_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Precision_score: %.2f' % precision_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
print('Roc_auc_score: %.2f' % roc_auc_score(Y_test.astype('int32'), y_pred.astype('int32'), average = 'micro'))
#gnb.score(X_train, Y_train.astype('int'))

Accuracy: 0.42
Recall: 0.42
F1_score: 0.42
Precision_score: 0.42
Roc_auc_score: 0.60


## 2. 聚类练习
对预处理后的census_income数据集中除最后一列的所有特征选择不同模型进行聚类，并进行结果比较和解释。

In [82]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import adjusted_rand_score, completeness_score, homogeneity_score

In [83]:
#KMeans聚类
X = X[:,:-2]
y_pred = KMeans(n_clusters = 2).fit_predict(X)

In [84]:
km = KMeans(n_clusters = 2).fit(X)
print('Adjusted_rand_score: %.5f' % adjusted_rand_score(X[:, -1], y_pred))
print('Completeness_score: %.5f' % completeness_score(X[:, -1], y_pred))
print('Homogeneity_score: %.5f' % homogeneity_score(X[:, -1], y_pred))

Adjusted_rand_score: -0.00002
Completeness_score: 0.00003
Homogeneity_score: 0.04774


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


In [86]:
#层次聚类
model = AgglomerativeClustering(distance_threshold = None, n_clusters = 2)
y_pred = model.fit_predict(X)

In [87]:
print('Adjusted_rand_score: %.5f' % adjusted_rand_score(X[:, -1], y_pred))
print('Completeness_score: %.5f' % completeness_score(X[:, -1], y_pred))
print('Homogeneity_score: %.5f' % homogeneity_score(X[:, -1], y_pred))

Adjusted_rand_score: 0.00062
Completeness_score: 0.00027
Homogeneity_score: 0.21500


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


In [88]:
#DBSCAN聚类
y_pred = DBSCAN(eps = 0.3, min_samples = 10).fit_predict(X)
print('Adjusted_rand_score: %.5f' % adjusted_rand_score(X[:, -1], y_pred))
print('Completeness_score: %.5f' % completeness_score(X[:, -1], y_pred))
print('Homogeneity_score: %.5f' % homogeneity_score(X[:, -1], y_pred))

Adjusted_rand_score: -0.00007
Completeness_score: 0.00000
Homogeneity_score: 0.00259


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)


### 3. 关联规则练习
利用data_generater读取“store_data.csv”并进行关联规则挖掘，找出支持度不小于0.0045，置信度不小于0.2的关联规则，并对结果进行解释。

In [1]:
from efficient_apriori import apriori

In [2]:
def data_generator(filename):
    def data_gen():
        with open(filename) as file:
            for line in file:
                yield tuple(k.strip() for k in line.split(','))
    return data_gen

In [3]:
transcations = data_generator('store_data.csv')
itemsets, rules = apriori(transcations, min_support = 0.0045, min_confidence = 0.2)

In [4]:
itemsets

{1: {('shrimp',): 536,
  ('cottage cheese',): 239,
  ('spinach',): 53,
  ('salad',): 37,
  ('avocado',): 250,
  ('green grapes',): 68,
  ('frozen smoothie',): 475,
  ('whole weat flour',): 70,
  ('salmon',): 319,
  ('antioxydant juice',): 67,
  ('energy drink',): 200,
  ('honey',): 356,
  ('tomato juice',): 228,
  ('vegetables mix',): 193,
  ('green tea',): 991,
  ('low fat yogurt',): 574,
  ('almonds',): 153,
  ('yams',): 86,
  ('mineral water',): 1788,
  ('olive oil',): 494,
  ('meatballs',): 157,
  ('burgers',): 654,
  ('eggs',): 1348,
  ('turkey',): 469,
  ('milk',): 972,
  ('energy bar',): 203,
  ('whole wheat rice',): 439,
  ('french fries',): 1282,
  ('whole wheat pasta',): 221,
  ('soup',): 379,
  ('shallot',): 58,
  ('light cream',): 117,
  ('spaghetti',): 1306,
  ('frozen vegetables',): 715,
  ('pet food',): 49,
  ('cookies',): 603,
  ('cooking oil',): 383,
  ('champagne',): 351,
  ('chicken',): 450,
  ('chocolate',): 1229,
  ('oil',): 173,
  ('extra dark chocolate',): 90,
  

In [5]:
rules

[{almonds} -> {green tea},
 {almonds} -> {mineral water},
 {avocado} -> {mineral water},
 {cottage cheese} -> {green tea},
 {cottage cheese} -> {mineral water},
 {energy drink} -> {mineral water},
 {frozen smoothie} -> {mineral water},
 {green tea} -> {mineral water},
 {tomato juice} -> {green tea},
 {honey} -> {mineral water},
 {low fat yogurt} -> {mineral water},
 {olive oil} -> {mineral water},
 {salmon} -> {mineral water},
 {shrimp} -> {mineral water},
 {tomato juice} -> {mineral water},
 {vegetables mix} -> {mineral water},
 {burgers} -> {eggs},
 {energy bar} -> {milk},
 {energy bar} -> {mineral water},
 {mineral water} -> {milk},
 {milk} -> {mineral water},
 {whole wheat rice} -> {milk},
 {whole wheat rice} -> {mineral water},
 {frozen vegetables} -> {spaghetti},
 {green tea} -> {spaghetti},
 {burgers} -> {mineral water},
 {cooking oil} -> {eggs},
 {cooking oil} -> {mineral water},
 {mineral water} -> {eggs},
 {eggs} -> {mineral water},
 {turkey} -> {eggs},
 {turkey} -> {mineral 

    1、购买各类物品的时候都会购买矿泉水，可能是因为矿泉水是必需品，且是烹饪的原材料
    2、购买高热量食物如hot dog，brownies等时，也会购买French fries等同类高热量食物
    3、巧克力会和各类食物一起购买，可能是因为巧克力属于小零食，一般摆放在收银台旁边，顾客会顺便购买
    4、需要一起烹饪的食物很可能会一起购买，比如cereal和milk，cooking oil和spaghetti，frozen vegetables和eggs等